In [1]:
!pip install SimpleITK
!git clone https://github.com/EPU-IA-2020/TP3.git


     |████████████████████████████████| 42.5MB 72kB/s 
Cloning into 'TP3'...
remote: Enumerating objects: 105090, done.
remote: Total 105090 (delta 0), reused 0 (delta 0), pack-reused 105090
Receiving objects: 100% (105090/105090), 1.77 GiB | 16.71 MiB/s, done.
Resolving deltas: 100% (29/29), done.
Checking out files: 100% (130744/130744), done.


In [0]:
import torch
import torch.nn as nn
import torch.utils.data
import torch.utils.tensorboard
import torchvision
import torchvision.transforms
import os
import time
import numpy as np
import skimage.transform
import matplotlib.pyplot as plt
import SimpleITK as sitk
import pandas as pd
import sklearn.model_selection as model_selection
from tqdm import tqdm

  # Part 1 : Study of the data

First, we will study the data. 

The data is stored in the folder /content/TP3/data.

For a patient BraTS19_EXAMPLE, you will have a folder /content/epu_ia_2019/BraTS19_EXAMPLE. Inside this folder you will have nifti files (.nii.gz) for eacch modalities and each z slice. 

The train, validation and test split are stored in the folder /content/TP3/datasets. For each split, you will have a text files with a list of patient

Execute the following code to :
*   Load the train, validation and test set
*   Print the first 5 patients of the train set
*   Print the lenght of the train, validation and test set


In [3]:
path = '/content/TP3/'
train_set = np.loadtxt(path + 'datasets/train.txt',dtype=str)
validation_set = np.loadtxt(path + 'datasets/val.txt',dtype=str)
test_set = np.loadtxt(path + 'datasets/test.txt',dtype=str)

# Train_set, validation_set and test_set are list of patients
print('Train set : {}'.format(train_set[:5])) # Print the first 5 patients of train_set
print('Train set lenght : {}'.format(len(train_set)))
print('Validation set lenght : {}'.format(len(validation_set)))
print('Test set lenght : {}'.format(len(test_set)))

Train set : ['BraTS19_CBICA_BHQ_1' 'BraTS19_CBICA_AQV_1' 'BraTS19_CBICA_ATN_1'
 'BraTS19_TCIA01_335_1' 'BraTS19_CBICA_ASY_1']
Train set lenght : 251
Validation set lenght : 42
Test set lenght : 42


Execute the following code to :
*   Print the first 5 patients in the data folder
*   Print the lenght of the data folder
*   Select a patient and print the number of images inside the patient folder
* Print the number of z slices for one patient
* Print the 5 first images of a patient

In [0]:
# The data is store in the folder /content/epu_ia_2019/data/
data_path = '/content/TP3/data/'
files = os.listdir(data_path) # All the files in the folder /content/epu_ia_2019/data/
print('Content of the folder : {}'.format(files[:5]))
print('Lenght of the folder : {}'.format(len(files)))

# For each patient we have 4 modality and the segmentation in the folder :
# t1, t2, flair, t1_ce (gado) and seg. 
# We also have a file for each slice along the z axis ()
patient = 'BraTS19_2013_0_1/'
patient_path = os.path.join(data_path, patient)
patient_files = os.listdir(patient_path)
print('Number of files for each patient : {}'.format(len(patient_files) ))
print('Number of slices : {}'.format(len(patient_files) / 5))
print('The first five files of the folder : {}'.format(patient_files[:5]))

##Exercice 1 : Study of medical images

Medical images are stored in specific data type such as dicom or nifti. 
Theses data types contain not only the images but also medical information. 
There are many Python librairy to load medical data. We will use SimpleITK. Use help in python in order to find the functions and their syntax


You should complete the following code using the functions sitk.GetDirection, sitk.GetOrigin, sitk.GetSpacing, sitk.GetMetaData, sitk.GetMetaDataKeys, sitk.ReadImage, sitk.GetArrayFromImage.

In [0]:
orig_data_path = '/content/TP3/origin_data/'

patients = ['BraTS19_CBICA_ANP_1', 'BraTS19_CBICA_AWV_1', 'BraTS19_TCIA01_131_1',
            'BraTS19_TCIA10_442_1']

modalities = ['_t1', '_t2', '_flair', 't1ce', 'seg']
suffix = '.nii.gz'
patient = patients[0]
modality = modalities[0]
patient_folder = os.path.join(orig_data_path,patient) 
# We use the librairy sitk to open the nifti images
image = '''CompleteHere'''(os.path.join(patient_folder, patient + modality + suffix))

# Print type of image
print('''CompleteHere''')

# Print geometrical information
print('Image Direction : {}'.format('''CompleteHere'''))
print('Image Spacing : {}'.format('''CompleteHere'''))
print('Image Origin : {}'.format('''CompleteHere'''))

print(image.GetPixel(0, 0, 0))

# Get all the information in the meta data
keys = '''CompleteHere'''
print('Metadata :')
for key in keys:
  print('{} : {}'.format('''CompleteHere'''))

# Convert the sitk image 
array = '''CompleteHere'''
print(type(array)) # Type of the image
print(array.shape)
print(array[0, 0, 0])

In order to accelerate the calculation time and have good results quickly, we will work on small images of shape (96, 96). The original images of the dataset have a shape (155, 240, 240) where the 155 correspond to the number of slice on the z axis.

The step to pass from the original images to the images we will use are :
- Crop the images to a shape of (155, 192, 192)
- Downsample the images by interpolation of scale 0.5 (https://scikit-image.org/docs/dev/auto_examples/transform/plot_rescale.html) to a shape of (77, 96, 96)
- Save all the z slice independantly in a new array of shape (96, 96)

Execute the following code to compare the original image and the preprocessed image.

Change the z parameter to plot different slice of the images 

In [0]:
orig_data_path = '/content/TP3/origin_data/'
patient = patients[0]
modality = modalities[0]
patient_folder = os.path.join(orig_data_path,patient) 
# We use the librairy sitk to open the nifti images
image = sitk.ReadImage(os.path.join(patient_folder, patient + modality + suffix))
orig_array = sitk.GetArrayFromImage(image)
print('Orig array shape : {}'.format(orig_array.shape))

###
data_path = '/content/TP3/data/'
patient_folder = os.path.join(data_path,patient)
z_slice = 35
path = os.path.join(patient_folder, patient + modality + '_z_' + str(z_slice) + suffix)
image = sitk.ReadImage(path)
processed_array = sitk.GetArrayFromImage(image)
print('Processed array shape : {}'.format(processed_array.shape))

plt.subplot(1, 2, 1)
plt.imshow(orig_array[z_slice*2, :, :], cmap='gray')
plt.title('Original array')
plt.subplot(1, 2, 2)
plt.imshow(processed_array, cmap='gray')
plt.title('Processed array')


[Solution](#scrollTo=GeuCzQ2m7PNy&line=27&uniqifier=1)


# Part 2 : Creation of the neural network

The network we will use is called UNet. It is famous network for medical image segmentation. In this part, we will create the network and train our first neural network. You can study the shape of the UNet in the figure 1 of the paper **U-Net: Convolutional Networks for Biomedical Image Segmentation** (https://arxiv.org/pdf/1505.04597.pdf). The UNet has two main features : First the size of the image is downsample by 2 in each block by a layer called **MaxPooling**. Secondly in order to keep information of high resolution, we use **skip-connection** to pass the information from the left part of the network to the right part.

## Exercice 2 : Create the network

- Complete the function DownConvBlock and UpConvBlock. You should use the different layers function from pytorch (Conv2D, BatchNorm2d, MaxPool2d, Upsample)
- Try to compare the following code with the figure 1 of the paper. Where are the DownConvBlock, the UpConvBlock ?


In [0]:
def get_activation(activation_type):
    activation_type = activation_type.lower()
    if hasattr(nn, activation_type):
      return getattr(nn, activation_type)()
    else:
      return nn.ReLU()

def _make_nConv(in_channels, out_channels, nb_Conv, activation='ReLU'):
  layers = []
  layers.append(ConvBatchNorm(in_channels, out_channels, activation))

  for _ in range(nb_Conv-1):
      layers.append(ConvBatchNorm(out_channels, out_channels, activation))
  return nn.Sequential(*layers)


class ConvBatchNorm(nn.Module):
  """(convolution => [BN] => ReLU)"""
  
  def __init__(self, in_channels, out_channels, activation='ReLU'):
    super(ConvBatchNorm, self).__init__()
    '''CompleteHere'''
      
  def forward(self, x):
    '''CompleteHere'''
    return 

class DownBlock(nn.Module):
  """Downscaling with maxpool convolution"""

  def __init__(self, in_channels, out_channels, nb_Conv, activation='ReLU'):
    super(DownBlock, self).__init__()
  '''CompleteHere'''
        
  def forward(self, x):
    '''CompleteHere'''
    return 

class UpBlock(nn.Module):
  """Upscaling then conv"""

  def __init__(self, in_channels, out_channels, nb_Conv, activation='ReLU'):
    super(UpBlock, self).__init__()
    '''CompleteHere'''

  def forward(self, x, skip_x):
    '''CompleteHere'''
    return

In [0]:
class UNet(nn.Module):
  def __init__(self, n_channels, n_classes):
    '''
    n_channels : number of channels of the input. 
                    By default 4, because we have 4 modalities
    n_labels : number of channels of the ouput.
                  By default 4 (3 labels + 1 for the background)
    '''
    super(UNet, self).__init__()
    self.n_channels = n_channels
    self.n_classes = n_classes

    self.inc = '''CompleteHere'''
    self.down1 = '''CompleteHere'''
    self.down2 = '''CompleteHere'''
    self.down3 = '''CompleteHere'''
    self.down4 = '''CompleteHere'''
    self.up1 = '''CompleteHere'''
    self.up2 = '''CompleteHere'''
    self.up3 = '''CompleteHere'''
    self.up4 = '''CompleteHere'''
    self.outc = '''CompleteHere'''
    self.last_activation = get_activation('Softmax')

  def forward(self, x):

    x1 = '''CompleteHere'''
    x2 = '''CompleteHere'''
    x3 = '''CompleteHere'''
    x4 = '''CompleteHere'''
    x5 = '''CompleteHere'''
    x = '''CompleteHere'''
    x = '''CompleteHere'''
    x = '''CompleteHere'''
    x = '''CompleteHere'''
    logits = self.last_activation(self.outc(x))
    return logits


## Exercice 3 : Study the model 

To study and debug a neural network, you can print the network and print the shape of the x inside the forward block.

Execute the following code and answer the following question.
- What are the number of convolutionnal layer in our model ?
- What is the number of parameters in our model ?
- What is the layer with the biggest number of parameters ? Try to find the formula to calculate the number of parameters.

In [0]:
model = UNet(n_channels=4, n_classes=4)

print(model)

# Image of size 96*96 with 4 modality + batch size = 1
x = torch.from_numpy(np.random.randn(1, 4, 96, 96)).float()
y = model(x)
print(y.shape)


torch.Size([1, 5, 96, 96])


## Exercice 4 : Dataset

This part is used to automatically load the correct images, put them in batch and applying transformation. We will also load the correct train, validation and test sets.

Complete the function **get_item** and **load** from the class **SegmentationDataset**

In [0]:
modalities = ['_t1', '_t1ce', '_t2', '_flair']
end = '.nii.gz'
seg_name = '_seg'

def load_split(split_folder):
    '''
        return train, val, test split with loadtxt
    '''
    train_split = np.loadtxt(os.path.join(
        split_folder, 'train.txt'), dtype=str)
    val_split = np.loadtxt(os.path.join(split_folder, 'val.txt'), dtype=str)
    test_split = np.loadtxt(os.path.join(split_folder, 'test.txt'), dtype=str)

    return train_split, val_split, test_split


def load_sitk(path):

    return sitk.GetArrayFromImage(sitk.ReadImage(path))


def find_z_slice(list_patient, threshold, dataframe):

    list_IDs = []

    for patient in list_patient:

        if threshold > 0:
          condition = dataframe[patient].values >= threshold
          z_slice = np.where(condition)[0]
        else:
          z_slice = range(155)
        list_IDs += list(set([(patient, int(z//2)) for z in z_slice]))

    return list_IDs


def generate_IDs(train_split, val_split, test_split,
                 tumor_percentage, csv_path, image_size=(240, 240)):

    tumor_volume_dataframe = pd.read_csv(csv_path)

    threshold = int(tumor_percentage * np.prod(image_size) / 100)

    train_IDs, val_IDs, test_IDs = [], [], []

    train_IDs = find_z_slice(train_split, threshold, tumor_volume_dataframe)
    val_IDs = find_z_slice(val_split, threshold, tumor_volume_dataframe)
    test_IDs = find_z_slice(test_split, 0, tumor_volume_dataframe)

    return train_IDs, val_IDs, test_IDs

class SegmentationDataset(torch.utils.data.Dataset):
  'Generates data for torch'

  def __init__(self, files_list, data_path, transform=None):
    'Initialization'
    super(SegmentationDataset, self).__init__()
    self.files_list = files_list
    self.transform = transform
    self.data_path = data_path

  def __len__(self):
    return len(self.files_list)

  def __getitem__(self, idx):
    'Generate one batch of data'
    ''' CompleteHere'''

    return (irm, mask, patient)

  def load(self, ID):

    patient, z_slice = ID
    patient_path = os.path.join(self.data_path, patient)

    irm = []
    for modality in modalities:
      path = os.path.join(patient_path,
                          '''CompleteHere''')
                          
      irm.append(load_sitk(path))

    irm = np.stack(irm, axis=0)

    mask_path = os.path.join(patient_path,
                              '''CompleteHere''')
    
    mask = load_sitk(mask_path)

    # Brats dataset has label 4 and not 3
    mask[mask == 4] = 3
    
    # One hot encoding
    label = 4
    mask = mask.astype(np.int16)
    mask = np.rollaxis(np.eye(label, dtype=np.uint8)[mask], -1, 0) 
      
    return irm, mask

**Utils** : group of useful functions to log information, put variable on cuda or on numpy

In [0]:
def to_var(x, device):
    if isinstance(x, np.ndarray):
        x = torch.from_numpy(x)
    x = x.to(device)
    return x

def to_numpy(x):
    if not (isinstance(x, np.ndarray) or x is None):
        if x.is_cuda: 
            x = x.data.cpu()
        x = x.numpy()
    return x

def save_checkpoint(state, save_path):
    '''
        Save the current model. 
        If the model is the best model since beginning of the training
        it will be copy
    '''

    if not os.path.isdir(save_path):
        os.makedirs(save_path)

    epoch = state['epoch']
    val_loss = state['val_loss']
    filename = save_path + '/' + \
        'model.{:02d}--{:.3f}.pth.tar'.format(epoch, val_loss)
    torch.save(state, filename)


def print_summary(epoch, i, nb_batch, loss, batch_time, 
                  average_loss, average_time, mode):
    '''
        mode = Train or Test
    '''
    summary = '[' + str(mode) + '] Epoch: [{0}][{1}/{2}]\t'.format(
        epoch, i, nb_batch)

    string = ''
    string += ('Dice Loss {:.4f} ').format(loss)
    string += ('(Average {:.4f}) \t').format(average_loss)
    string += ('Batch Time {:.4f} ').format(batch_time)
    string += ('(Average {:.4f}) \t').format(average_time)

    summary += string

    print(summary)


**Tensorboard** : This part is used to plot the prediction of the network during the training and study its performance.

In [0]:
def plot(irms, masks=None, pred_masks=None):
    
    kwargs = {'cmap': 'gray'}

    fig, ax = plt.subplots(2, 3, gridspec_kw={'wspace': 0.15, 'hspace': 0.2,
                                              'top': 0.85, 'bottom': 0.1,
                                              'left': 0.05, 'right': 0.95})
        
    ax[0, 0].imshow(irms[0, :, :], **kwargs)
    
    if masks is not None:
        masks = np.argmax(masks, axis=0)
        ax[0, 1].imshow(masks, vmin=0, vmax=3)
        
    if pred_masks is not None:
        pred_masks = np.argmax(pred_masks, axis=0)
        ax[0, 2].imshow(pred_masks, vmin=0, vmax=3)

    for i in range(3):
        ax[1, i].imshow(irms[i+1, :, :], **kwargs)

    for i in range(2):
        for j in range(3):
            ax[i, j].grid(False)
            ax[i, j].axis('off')
            ax[i, j].set_xticks([])
            ax[i, j].set_yticks([])

    ax[0, 0].set_title('IRM T1')
    ax[1, 0].set_title('IRM Gado')
    ax[1, 1].set_title('IRM T2')
    ax[1, 2].set_title('IRM Flair')
    
    ax[0, 1].set_title('Ground Truth Seg')
    ax[0, 2].set_title('Predicted Seg')

    fig.canvas.draw()
    
    return fig



**Loss** : This code is used to code the dice loss which is a loss function dedicated to medical segmentation. 

In [0]:
def dice_loss(input, target):
    smooth = 1.
    target = target.float()
    input = input.float()
    input_flat = input.contiguous().view(-1)
    target_flat = target.contiguous().view(-1)
    intersection = (input_flat * target_flat).sum()
    return 1 - ((2. * intersection + smooth) /
                (input_flat.pow(2).sum() + target_flat.pow(2).sum() + smooth))

def mean_dice_loss(input, target):
  
  channels = list(range(target.shape[1]))
  loss = 0
  for channel in channels:
    dice = dice_loss(input[:, channel, ...],
                     target[:, channel, ...])
    loss += dice

  return loss / len(channels)


## Exercice 5 : Main - Train the model 

To train the model in pytorch, you have different steps :
- Choose the optimiser, learning rate and loss function
- Create the model
- Define the data by using the **Dataset** and **DataLoader** class
- Implement the train loop
- Launch the training for a number of epochs

Complete the following code and launch a training for a small number of epochs (5 or 10 for instance). 
You should see the loss decreasing. 
Then open tensorboard to visualise the decrease of the loss and the prediction of the network at the different epochs


You need to complete the **train_loop** function and the **MAIN** block

In [0]:
# Import of the package and choice of the parameters

####### Parameters of models #######
learning_rate = 1e-4
image_size = (96, 96)
n_modality = 4
n_labels = 4
epochs = 10
batch_size = 64
print_frequency = 5
save_frequency = 10
save_model = True
tumor_percentage = 0.5
tensorboard = True
main_path = '/content/TP3/'
sets_path = os.path.join(main_path, 'datasets/')
csv_path = os.path.join(main_path, 'data/tumor_count.csv')
data_folder = os.path.join(main_path, 'data/')
save_path = '/content/save/'
loss_function = 'dice_loss'
session_name = 'Test_session' + '_' + time.strftime('%m.%d %Hh%M')
model_path = save_path + 'models/' + session_name + '/'
tensorboard_folder = save_path + 'tensorboard_logs/'

In [0]:
# Load the split, generate the IDs list and create the DataGenerator
train_split, val_split, test_split = load_split(sets_path)
(train_IDs, val_IDs,
 test_IDs) = generate_IDs(train_split, val_split, test_split,
                          tumor_percentage, csv_path)

transformation=None
train_Dataset = SegmentationDataset(train_IDs, data_path=data_folder,
                                    transform=transformation
                                    )

val_Dataset = SegmentationDataset(val_IDs, data_path=data_folder,
                                  transform=transformation
                                  )

test_Dataset = SegmentationDataset(val_IDs, data_path=data_folder,
                                  transform=transformation
                                  )
        
train_loader = torch.utils.data.DataLoader(train_Dataset, 
                                           batch_size=batch_size, shuffle=True,
                                           drop_last=True)

val_loader = torch.utils.data.DataLoader(val_Dataset,
                                         batch_size=batch_size, drop_last=True) 

test_loader = torch.utils.data.DataLoader(val_Dataset,
                                         batch_size=1, drop_last=False)    

In [0]:
# Train the model
# Complete the train loop 

def train_loop(loader, model, criterion, optimizer, writer, epoch):

    logging_mode = 'Train' if model.training else 'Val'
    
    end = time.time()
    time_sum, loss_sum = 0, 0
    
    for i, sample in enumerate(loader, 1):
        # Take the variables and put them to GPU
        ('''CompleteHere''') = sample
        
        '''CompleteHere''' = to_var('''CompleteHere'''.float(), device)
        '''CompleteHere''' = to_var('''CompleteHere'''.float(), device)

        # print(irms.shape) # Batch * Modality * Width * Height
    
        # compute output
        '''CompleteHere'''

        # compute loss
        '''CompleteHere'''

        # compute gradient and do SGD step
        if model.training:
            '''CompleteHere'''
        
        # measure elapsed time
        batch_time = time.time() - end

        time_sum += batch_size*batch_time
        loss_sum += batch_size*dice_loss
        average_loss = loss_sum / (i * batch_size)
        average_time = time_sum / (i * batch_size)

        end = time.time()
        
        if i % print_frequency == 0:
            print_summary(epoch + 1, i, len(loader), dice_loss, batch_time,
                          average_loss, average_time, logging_mode)
        if tensorboard:
            step = epoch*len(loader) + i
            writer.add_scalar(logging_mode + '_dice', dice_loss.item(),step)

    if tensorboard:
      n = irms.shape[0]
      irms = to_numpy(irms)
      masks = to_numpy(masks)
      pred_masks = to_numpy(pred_masks)

      for batch in range(n):
        fig = plot(irms[batch, ...], 
                   masks[batch, ...], pred_masks[batch, ...])
        
        writer.add_figure(logging_mode + str(batch), 
                          fig, epoch)

    return dice_loss

In [0]:
### MAIN ###

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = UNet(n_channels=n_modality, n_classes=n_labels) # Create model
model.to(device) # move model to the right device

criterion = '''CompleteHere''' # Choose loss function
optimizer = '''CompleteHere'''# Choose optimize
    
if tensorboard:
    log_dir = tensorboard_folder + session_name + '/'
    if not os.path.isdir(log_dir):
        os.makedirs(log_dir)
    writer = torch.utils.tensorboard.SummaryWriter(log_dir)
else:
    writer = None

for epoch in range(epochs):  # loop over the dataset multiple times
    print('******** Epoch [{}/{}]  ********'.format(epoch+1, epochs+1))
    print(session_name)

    # train for one epoch
    model.train()
    print('Training')
    train_loop(train_loader, model, criterion, optimizer, writer, epoch)

    # evaluate on validation set
    print('Validation')
    with torch.no_grad():
        model.eval()
        val_loss = train_loop(val_loader, model, criterion,
                              optimizer, writer, epoch)

    if save_model and epoch % save_frequency == 0:
        save_checkpoint({'epoch': epoch,
                        'state_dict': model.state_dict(),
                         'val_loss': val_loss,
                         'optimizer': optimizer.state_dict()}, model_path)
        

In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir '/content/save/tensorboard_logs/'

# Part 3 : Prediction and Evaluation

After training the model we have to perform two steps : the prediction of the segmentation using a model trained and the evaluation of the performance of the network.

The evaluation of the network should be done on the validation set (which have not been seen by the network during the training). The test set will only be used when all the parameters of the network are chosen. 

## Exercice 6 : Prediction and returning to original space

In this exercice, we will load a network and use the function **predict()** to calculate the predicted segmentation. The predicted segmentation will have a shape (96, 96, 4). We need to transform it back to the 3D shape (155, 240, 240) to perform the evaluation. 
The steps to calculate the prediction will be : 

- Apply **predict()** function on a dataloader, with a model. We don't need to calculate the loss and backpropagate them. 
- Apply **reconstruct_patient()** to find and save in the good order all the slice corresponding to one patient.
- Apply **get_mask2original_shape()** to the prediction to go from a shape (78,4,96,96) to (155,240,240)

**Questions**
- Complete the **predict()** function



In [0]:
train_split, val_split, test_split = load_split(sets_path)
(train_IDs, val_IDs,
 test_IDs) = generate_IDs(train_split, val_split, test_split,
                          0, csv_path)

transformation=None
test_Dataset = SegmentationDataset(test_IDs, data_path=data_folder,
                                  transform=transformation
                                  )

test_loader = torch.utils.data.DataLoader(test_Dataset,
                                         batch_size=batch_size, drop_last=False) 

test_path = '/content/TP3/origin_data/'
data_path = '/content/TP3/data/'
patients = os.listdir(test_path)
patient = patients[0]
z_max = 78

In [0]:
def get_mask2original_shape(predict_mask):
  '''
    input shape is 77*4*96*96
  '''
  mask = np.zeros(shape=(155,240, 240))
  res = skimage.transform.resize(predict_mask, (155, 4, 192, 192))
  print('Shape after rescale : {}'.format(res.shape))
  res = res > 0.5
  res = np.argmax(res, axis=1)
  mask[:, 24:-24, 24:-24] = res
  mask[mask == 3] = 4
  return mask.astype('int')


In [0]:
def predict(loader, model):
    preds = {}
    for i, sample in tqdm(enumerate(loader, 1)):
        # Take variable and put them to GPU
        (irms, masks, patients) = '''CompleteHere'''
        irms = '''CompleteHere'''
        masks = '''CompleteHere'''
    
        # compute output
        pred_masks = '''CompleteHere'''

        # Put the predictions in a dictionnary with one key being the 
        # patient and the z slice
        n = pred_masks.shape[0]
        for j in range(n):
          patient, z_slice = patients[0][j], patients[1][j]
          name = patient + '_z_' + str(to_numpy(z_slice))
          preds[name] = to_numpy(pred_masks[j])
    return preds

def reconstruct_patient(preds, patient):
  '''
    From the dictionnary with prediction find the slice corresponding to 
    one patient and construct a 3D matrix of shape 77*96*96
  '''
  X = []
  for i in range(z_max):

    name = patient + '_z_' + str(i)
    array_slice = preds[name]
    X.append(array_slice)

  X = np.stack(X, axis=0) 
  return X


In [0]:
# Do the prediction for the test loader
model.eval()
with torch.no_grad():
  preds = predict(test_loader, model, batch_size=batch_size)


In [0]:
# Reconstruct the prediction for one patient and get to its original shape
predict_mask = reconstruct_patient(preds, patient)
print(predict_mask.shape)
predict_mask = get_mask2original_shape(predict_mask)
print(predict_mask.shape)

## Exercice 7 : Visual Comparaison

We will do a visual comparaison between the predicted mask and the groundtruth. We will perform the comparaison with **Matplotlib** (python librariry for graph) and a software specialised for medical imaging vizualisation **3D Slicer** (https://www.slicer.org/)

**Questions**
- Load the groundtruth mask (in folder /content/TP3/origin_data) and using the function **plt.subplot**, **plt.imshow** and **plt.title**, plot the predicted mask and the groundtruth side by side
- Change the value of the **z_slice** parameter to explore the segmentation slice by slice
- Study the function **numpy2nifti**. What does it do ? Apply it and save the predicted mask as a nifti files. Download the MRI, the groundtruth segmentation and the predicted segmentation and download the software 3D Slicer. Open the nifti files in 3D Slicer and compare the images.

In [0]:
# Compare visual plot of prediction and original mask 
import matplotlib.pyplot as plt

patient_folder = os.path.join(test_path,patient)
orig_image = '''CompleteHere'''(os.path.join(patient_folder, 
                                    patient  + '_seg' + suffix))
orig_mask = '''CompleteHere'''(orig_image)

z_slice=100

plt.subplot(1,2,1)
'''CompleteHere'''('''CompleteHere''', vmin=0, vmax=4)
'''CompleteHere'''('Original mask (slice {})'.format(z_slice))

plt.subplot(1,2,2)
'''CompleteHere'''('''CompleteHere''', vmin=0, vmax=4)
'''CompleteHere'''('Predicted mask (slice {})'.format(z_slice));

In [0]:
def numpy2sitk(predict_mask, orig_img):
  '''
    Input : predict_mask of type numpy array
            orig_img of type SimpleITK image
    Output : new_img of type SimpleITK image 
  '''
  new_img = sitk.GetImageFromArray(predict_mask)
  new_img.SetDirection(orig_img.GetDirection())
  new_img.SetOrigin(orig_img.GetOrigin())
  new_img.SetSpacing(orig_img.GetSpacing())
  return new_img

predict_img = numpy2sitk(predict_mask, orig_image)
path = os.path.join(patient_folder, patient + '_predict_seg' + suffix)
sitk.WriteImage(predict_img, path) # Save the predictions as nifti images

## Exercice 8 : Metrics 

We will now calculate the evaluation metrics to assess the performance of our model. We will use 3 metrics : the sensitivity (also called true positive rate), the Specificity (also called true negative rate) and the dice score (different of the dice loss used during the training). 

We will evaluate the metrics on the 5 patients in the folder /content/TP3/origin_data. In the reality, we should evaluate the metrics for all the patient of the validation set (the test set will only be used when we choose the best model and we publish our results).

**Questions :**
- Complete the **metrics()** function in order to calculate the Dice, Sensitivity and Specificity.

- Using, a for loop and all the functions defined in the part 3, calculate the metrics for the 5 patients in the folder /content/TP3/origin_data. You should print the average Dice value for each category (WT, ET, TC)

In [0]:
import pandas as pd

def metrics(mask_, gt_):
  '''
    Taking to binary array of same shape as input
    This function compute the confusion matrix and use it to calculate 
    Dice metrics, Sensitivity and Specificity
    Input : mask_, gt_ numpy array of identic shape (only 1 and 0)
    Output : List of 3 scores
  '''
  lnot = np.logical_not
  land = np.logical_and

  true_positive = '''CompleteHere'''
  false_positive = '''CompleteHere'''
  false_negative = '''CompleteHere'''
  true_negative = '''CompleteHere'''

  M = np.array([[true_negative, false_negative],
                [false_positive, true_positive]]).astype(np.float64)
  metrics = {}
  metrics['Sensitivity'] = '''CompleteHere'''
  metrics['Specificity'] = '''CompleteHere'''
  metrics['Dice'] = '''CompleteHere'''

  return [metrics['Dice'], metrics['Sensitivity'], metrics['Specificity']]


def evalAllSample(mask_, gt_):
  '''
    This functions takes as input two numpy array with labels between
    0, 1, 2 and 4 and calculate the metrics as defined in BraTS data challenge
    mask_ and gt_ should be array of int
  '''
  # whole tumor (labels 1,2,4)
  mask_wt, gt_wt = (np.array([0, 1, 1, 0, 1])[mask_], 
                    np.array([0, 1, 1, 0, 1])[gt_])
  wt_metrics = metrics(mask_wt, gt_wt)

  # tumor core (labels 1,4)
  mask_tc, gt_tc = (np.array([0, 1, 0, 0, 1])[mask_], 
                    np.array([0, 1, 0, 0, 1])[gt_])
  tc_metrics = metrics(mask_tc, gt_tc)

  # enhancing tumor (label 4)
  mask_et, gt_et = (np.array([0, 0, 0, 0, 1])[mask_], 
                    np.array([0, 0, 0, 0, 1])[gt_])
  et_metrics = metrics(mask_et, gt_et)
  
  return pd.DataFrame({'wt': wt_metrics, 'tc': tc_metrics, 'et': et_metrics},
                      index=['Dice', 'Sensitivity', 'Specificity'])



In [0]:
wt_dice_list = []
et_dice_list = []
tc_dice_list = []

model.eval()
with torch.no_grad():
  preds = predict(test_loader, model, batch_size=batch_size)

for patient in patients:

  patient_folder = os.path.join(test_path,patient)
  # load the sitk image and pass it into numpy array
  orig_image = '''CompleteHere'''
  orig_mask = '''CompleteHere'''
  
  # Reconstruct the prediction and pass it into the original shape
  predict_mask = '''CompleteHere'''
  predict_mask = '''CompleteHere'''
  
  print('*********** {} ***********'.format(patient))
  # Calculate scores
  scores = '''CompleteHere'''
  print(scores)
  print()

  '''CompleteHere'''.append(scores.loc['Dice', 'wt'])
  '''CompleteHere'''.append(scores.loc['Dice', 'et'])
  '''CompleteHere'''.append(scores.loc['Dice', 'tc'])


In [0]:
print('Whole Tumor Dice : {:.2f}'.format(np.mean(wt_dice_list)))
print('Tumor Core Dice : {:.2f}'.format(np.mean(et_dice_list)))
print('Enhancing Tumor Dice : {:.2f}'.format(np.mean(tc_dice_list)))

# Part 4 : Data Augmentation

In this part we will use data augmentation to increase the performance of the network. The idea with data augmentation is to create new artificial sample so that the network see more data and overfit less.




## Exercice 8 : Study of the data augmentation

Complete each of the following code to see the impact of the data augmentation. Compare the original data and the new artificial images.

You need to complete the following class :
AxialFlip, RandomCrop and the 3 functions getZoomMatrix, getRotationMatrix, getTranslationMatrix

In [0]:
import scipy
import scipy.ndimage
import numpy as np

class AxialFlip(object):

    def __call__(self, sample):

        choice_x = np.random.randint(0, 2)
        choice_y = np.random.randint(0, 2)
        
        irm, mask = sample
        new_sample = (self.axialflip(irm, choice_x, choice_y),
                      self.axialflip(mask, choice_x, choice_y))

        return new_sample

    def axialflip(self, array, choice_x, choice_y):
      '''
        Shape of array is Modality * H*W
      '''
        if choice_x == 1:
          '''CompleteHere'''

        if choice_y == 1:
          '''CompleteHere'''

        return np.ascontiguousarray(array)

class RandomRotation90(object):
    '''
        Taken from augment_rot90 from MIC-DKFZ/batchgenerators
        https://github.com/MIC-DKFZ/batchgenerators/blob/master/batchgenerators/augmentations/spatial_transformations.py
    '''

    def __init__(self, num_rot=(1, 2, 3, 4)):

        self.num_rot = num_rot
        self.axes = (1,2)

    def __call__(self, sample):
        '''
          irm and mask have shape (Modality*Width*Height) and (Label*Width*Height)
        '''
        num_rot = np.random.choice(self.num_rot)
        
        def f(img):
            return np.ascontiguousarray(np.rot90(img, num_rot, self.axes))
        
        irm, mask = sample
        new_sample = (f(irm), f(mask))

        return new_sample

class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, cubic crop
            is made.
    """

    def __init__(self, output_size, dim=2):
        assert isinstance(output_size, (int, tuple, list))
        if isinstance(output_size, int):
            self.output_size = dim * (output_size,)
        else:
            assert len(output_size) == dim
            self.output_size = output_size  

    def __call__(self, sample):
        
        irm, mask = sample
        
        _, height, width = irm.shape
    
        i = '''CompleteHere'''
        j = '''CompleteHere'''

        def f(img):
            return img['''CompleteHere''']

        new_sample = ( f(irm), f(mask))

        return new_sample

def RandomTranslation(max_translation=30, transform_matrix=None, debug=False):
    
    translation = np.random.randint(-max_translation, max_translation, 2)
    
    if debug:
        return getTranslationMatrix(translation, transform_matrix), (translation)
    else:
        return getTranslationMatrix(translation, transform_matrix)
    
def RandomRotation(theta_max=20, transform_matrix=None, debug=False):
    
    theta = np.random.uniform(-theta_max, theta_max)

    if debug:
        return getRotationMatrix(theta, transform_matrix), theta
    else:
        return getRotationMatrix(theta, transform_matrix)

def RandomZoom(zoom_max=0.2, transform_matrix=None, debug=False):
  
  zoom = np.random.uniform(1 - zoom_max, 1 + zoom_max)

  if debug:
      return getZoomMatrix(zoom, transform_matrix), zoom
  else:
      return getZoomMatrix(zoom, transform_matrix)

def getTranslationMatrix(translation, transform_matrix=None):
    '''
        2D translation on the axis (0, 1). 
        Axis 3 is the modality axis
        tx: Width shift.
        ty: Heigh shift.
    
    '''
    shift_matrix = '''CompleteHere'''

    if transform_matrix is None:
        transform_matrix = shift_matrix
    else:
        transform_matrix = np.dot(transform_matrix, shift_matrix)
            
    return transform_matrix

def getZoomMatrix(zoom, transform_matrix=None):
    '''
        Affine Zoom in 2D
        zx: Zoom in x direction.
        zy: Zoom in y direction
    '''
    zoom_matrix = '''CompleteHere'''
    if transform_matrix is None:
        transform_matrix = zoom_matrix
    else:
        transform_matrix = np.dot(transform_matrix, zoom_matrix)
            
    return transform_matrix

def getRotationMatrix(theta, transform_matrix=None):
    '''
        2D rotation on the axis (0, 1). 
        Axis 3 is the modality axis
        theta: Rotation angle in degrees.
    '''
    theta = np.deg2rad(theta)
    rotation_matrix = '''CompleteHere'''
    
    if transform_matrix is None:
        transform_matrix = rotation_matrix
    else:
        transform_matrix = np.dot(transform_matrix, rotation_matrix)
    
    return transform_matrix


def apply_affine_transform(x, seg=None, transform_matrix=None, crop_shape=None, 
                           fill_mode='nearest', cval=0., order=3):
    """Applies an affine transformation specified by the parameters given.
    # Arguments
        x: 4D numpy array, single image, multimodalities (Modality*H*W)
        fill_mode: Points outside the boundaries of the input
            are filled according to the given mode
            (one of `{'constant', 'nearest', 'reflect', 'wrap'}`).
        cval: Value used for points outside the boundaries
            of the input if `mode='constant'`.
        order: int, order of interpolation
    # Returns
        The transformed version of the input.
    """
    if scipy is None:
        raise ImportError('Image transformations require SciPy. '
                          'Install SciPy.')
    if transform_matrix is not None:
        
        channels, h, w = x.shape
                
        transform_matrix = transform_matrix_offset_center(transform_matrix, 
                                                          h, w)

        res = [ scipy.ndimage.affine_transform(x[channel, ...], transform_matrix,
                                              order=order, mode=fill_mode, cval=cval)  for channel in range(channels)]
        x = np.stack(res, axis=0)
        
        if seg is not None:
            
            labels = seg.shape[0]
            res = [scipy.ndimage.affine_transform(seg[label, ...], transform_matrix,
                                                                order=order, mode=fill_mode, cval=cval) for label in range(labels)]
            
            seg = np.stack(res, axis=0)
            seg[seg > 0.5] = 1
            seg[seg < 0.5] = 0
            
            return x, seg 
        
    return x

def transform_matrix_offset_center(matrix, x, y):
    o_x = float(x) / 2 + 0.5
    o_y = float(y) / 2 + 0.5
    offset_matrix = np.array([[1, 0, o_x], 
                              [0, 1, o_y],
                              [0, 0, 1]])
    
    reset_matrix = np.array([[1, 0, -o_x], 
                             [0, 1, -o_y],
                             [0, 0, 1]])
    
    transform_matrix = np.dot(np.dot(offset_matrix, matrix), reset_matrix)
    
    return transform_matrix

class AffineTransform(object):
    
    def __init__(self, theta=0, max_translation=0, max_zoom=0):

        
        self.theta = theta
        self.max_translation = max_translation
        self.max_zoom = max_zoom
        
    def __call__(self, sample):
        
        transform_matrix = np.eye(3)

        if self.theta > 0:
            transform_matrix = RandomRotation(self.theta)

        if self.max_translation > 0:
            transform_matrix = RandomTranslation(self.max_translation,
                                                 transform_matrix)

        if self.max_zoom > 0:
            transform_matrix = RandomZoom(self.max_zoom,
                                          transform_matrix)

        new_irm, new_mask = apply_affine_transform(irm, mask, transform_matrix)

        return (new_irm, new_mask)


In [0]:
k = 3

irms, masks, patient = val_loader.__iter__().next()

# irms is a list of a numpy array of shape [Batch * W*H*Modality]
irm = irms[0,:].numpy()
mask = masks[0,:].numpy()

fig = plot(irm, mask)
fig.suptitle('Original image')
fig.savefig(save_path + 'orig.png')

Axial Flip 

In [0]:
'''CompleteHere'''
for i in range(k):

    (new_irm, new_mask) = '''CompleteHere'''((irm, mask))

    fig = plot(new_irm, new_mask)
    fig.suptitle('Random Axial Flip')
    fig.savefig(save_path + 'Axial_Flip{}.png'.format(k))

90° Rotation 

In [0]:
'''CompleteHere'''
for i in range(k):

    (new_irm, new_mask) = '''CompleteHere'''((irm, mask))

    fig = plot(new_irm, new_mask)
    fig.suptitle('Random 90° Rotation')
    fig.savefig(save_path + 'Random90Rotation{}.png'.format(k))

In [0]:
'''CompleteHere'''
for i in range(k):

    (new_irm, new_mask) = '''CompleteHere'''((irm, mask))

    fig = plot(new_irm, new_mask)
    fig.suptitle('Random crop')
    fig.savefig(save_path + 'Random90Rotation{}.png'.format(k))

Random Rotation with +/- 30°

In [0]:
'''CompleteHere'''
for i in range(k):
    

    (new_irm, new_mask) = '''CompleteHere'''((irm, mask))

    fig = plot(new_irm, new_mask)
    fig.suptitle('Random rotation')
    fig.savefig(save_path + 'Rotation{:.1f}.png'.format(i))


Random Translation 

In [0]:
'''CompleteHere'''

for i in range(k):
    new_irm, new_mask  = '''CompleteHere'''((irm, mask))

    fig = plot(new_irm, new_mask)
    fig.suptitle('Random translation')
    fig.savefig(save_path + 'Translation_{}.png'.format(i))

Random Zoom

In [0]:
'''CompleteHere'''

for i in range(k):
    (new_irm, new_mask)  = '''CompleteHere'''((irm, mask))

    fig = plot(new_irm, new_mask)
    fig.suptitle('Random zoom ')
    fig.savefig(save_path + 'Zoom_{}.png'.format(k))

All transformations

In [0]:
'''CompleteHere'''

for i in range(k):
    (new_irm, new_mask)  = '''CompleteHere'''((irm, mask))

    fig = plot(new_irm, new_mask)
    fig.suptitle('Rotation Translation Zoom')
    fig.savefig(save_path + 'All_transforms{}.png'.format(i))

## Exercice 9 : Train with data augmentation

Relaunch a new training with data augmentation. The training will take more time than during the previous training.

In [0]:
transforms_list = [AffineTransform(theta=15, max_translation=15, max_zoom=0.3),
                   AxialFlip(), RandomRotation90(), RandomCrop((64, 64))
                   ]

transformation = torchvision.transforms.Compose(transforms_list)
train_Dataset = SegmentationDataset(train_IDs, data_path=data_folder,
                                    transform=transformation
                                    )

val_Dataset = SegmentationDataset(val_IDs, data_path=data_folder,
                                  transform=transformation
                                  )

train_loader_data_augment = torch.utils.data.DataLoader(train_Dataset, 
                                           batch_size=batch_size, shuffle=True,
                                           drop_last=True)

val_loader_data_augment = torch.utils.data.DataLoader(val_Dataset,
                                         batch_size=batch_size, drop_last=True) 


In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = UNet(n_channels=4, n_classes=4) # Create model
model.to(device) # move model to the right device

criterion = mean_dice_loss # Choose loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # Choose optimize
    
if tensorboard:
    log_dir = tensorboard_folder + session_name + '/'
    if not os.path.isdir(log_dir):
        os.makedirs(log_dir)
    writer = torch.utils.tensorboard.SummaryWriter(log_dir)
else:
    writer = None

for epoch in range(epochs):  # loop over the dataset multiple times
    print('******** Epoch [{}/{}]  ********'.format(epoch+1, epochs+1))
    print(session_name)

    # train for one epoch
    model.train()
    print('Training')
    train_loop(train_loader_data_augment, model, criterion,
               optimizer, writer, epoch)

    # evaluate on validation set
    print('Validation')
    with torch.no_grad():
        model.eval()
        val_loss = train_loop(val_loader_data_augment, model, criterion,
                              optimizer, writer, epoch)

    if save_model and epoch % save_frequency == 0:
        save_checkpoint({'epoch': epoch,
                        'state_dict': model.state_dict(),
                         'val_loss': val_loss,
                         'optimizer': optimizer.state_dict()}, model_path)
        

## Exercice 10 : Comparison of training curves

Two models trained for 200 epochs are given in the folder /content/TP3/tensorboard_logs/. The only difference between the two models in that one is using data augmentation and the other not. 
Open tensorboard and compare the loss, accuracy and prediction for the two models.


In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir /content/TP3/tensorboard_logs/